# Advanced Movie/Music Editing

If you want to do fancy editing with movies and sound, you probably need more serious software. This will vary by platform, but for movies on Mac, iMovie is a good basic option that comes with most new machines, Final Cut Pro has more options. For music, Garage Band does a lot of basic things, Logic has more functionality. If trying to combine a MIDI in your movie, it's probably not worth doing anything fancier than line them up by hand in the software.

# Command Line Editing

There are also a number of open-source, very capable packages that allow you to do your editing on the command line. 

**DISCLAIMER: This will not work for everyone, and we can't hope to effectively troubleshoot people's platform/dependency issues.** The various libraries have great documentation. We can tell you what works for us on Mac, which you can feel free to use as a starting point. We use `timidity` for manipulating the MIDI files, and `ffmpeg` for the movies.

# Installation

The packages can both be installed using homebrew. For FFMPEG you have to make sure you install the codecs you want to use:

```
brew install timidity
$ brew install ffmpeg --with-fdk-aac --with-ffplay --with-freetype --with-libass --with-libquvi --with-libvorbis --with-libvpx --with-opus --with-x265
```

# Making a Movie

We first have to make the frames, as described in [MakingAMovie.ipynb](MakingAMovie.ipynb). Let's load and run that notebook to generate the frames that we will need to stitch together:

In [1]:
%run ./MakingAMovie.ipynb

CPU times: user 17 ms, sys: 22.4 ms, total: 39.4 ms
Wall time: 4.43 s


You can also run these commands on the command line. From the notebook, we have to make sure we delete any file with the same name or the command will fail. We also have to specify the frames per second we used in generating the frame events, which defaults to 30:

In [2]:
moviename = 'test.mp4'
fps = 30
try:
    call("rm -f {0}".format(moviename), shell=True)
except:
    pass
call("ffmpeg -r {0} -i tmp/pngs/%05d.png -c:v libx264 -pix_fmt yuv420p {1}".format(fps, moviename), shell=True)
call("open test.mp4", shell=True)

0

# Combining a movie with a MIDI

Let's first make a simple MIDI (see [WritingMIDIFiles.ipynb](WritingMIDIFiles.ipynb)) that we want to add in our movie:

In [3]:
from midiutil import MIDIFile
import sys
sys.path.append('../')
import systemsounds as ss

bpm = 30
midifile = MIDIFile(adjust_origin=True)
midifile.addTempo(track=0, time=0, tempo=bpm) 
transit_notes = ss.calc_midi_notes(sim.particles, ref_note=48, ref_ID=-1)
conjunction_note = 72 # play higher note at conjunctions (C5)

for transit in transits.events:
    if transit['target'] >= 5:
        midifile.addNote(track=0, channel=transit['target'], pitch=transit_notes[transit['target']], time=transit['time'], duration=1, volume=100)
for conjunction in conjunctions.events:
    if conjunction['target'] >= 5:
        midifile.addNote(track=0, channel=sim.N, pitch=conjunction_note, time=conjunction['time'], duration=1, volume=100)

with open("test.mid", "wb") as f:
    midifile.writeFile(f)

We first have to convert our MIDI into a wav file, and we make sure the initial silence is not cut off (which would get things off sync). The MIDI also includes silence at the end, which makes it longer than our movie, so we then use `ffmpeg` to trim it to the right length. For this we need the length of the movie in seconds, which we can calculate from the last frame event:

In [4]:
lastevent = frames.events[-1]
elapsed_time = float(lastevent['frame_ctr'])/lastevent['fps']
try:
    call("rm -f test.mp4", shell=True)
    call("rm -f tmp/test.wav", shell=True)
    call("rm -f tmp/testcut.wav", shell=True)
except:
    pass
call("timidity -Ow test.mid -o tmp/test.wav --preserve-silence", shell=True)
call("ffmpeg -t {0} -i tmp/test.wav tmp/testcut.wav".format(elapsed_time), shell=True)

0

Finally we stitch our frames together and include the MIDI:

In [5]:
call("ffmpeg -r {0} -i tmp/pngs/%05d.png -i tmp/testcut.wav -c:v libx264 -pix_fmt yuv420p -c:a libfdk_aac -b:a 192k -shortest test.mp4".format(fps), shell=True)
call("open test.mp4", shell=True)

0